In [11]:
pip uninstall aiobotocore -y


Found existing installation: aiobotocore 2.19.0
Uninstalling aiobotocore-2.19.0:
  Successfully uninstalled aiobotocore-2.19.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip uninstall boto3 botocore s3transfer -y


Found existing installation: boto3 1.40.46
Uninstalling boto3-1.40.46:
  Successfully uninstalled boto3-1.40.46
Found existing installation: botocore 1.40.46
Uninstalling botocore-1.40.46:
  Successfully uninstalled botocore-1.40.46
Found existing installation: s3transfer 0.14.0
Uninstalling s3transfer-0.14.0:
  Successfully uninstalled s3transfer-0.14.0


In [5]:
import json
import requests
import io
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from docx import Document
from docx.shared import Inches, Pt, RGBColor
import os
from datetime import datetime
import time

    # Initialize AWS client
    secrets_client = boto3.client('secretsmanager')

    # Email Configuration
    SMTP_SERVER = "smtp.gmail.com"
    SMTP_PORT = 587
    SENDER_EMAIL = "macheoreports@macheo.org"
    RECIPIENT_EMAIL = "macheoreports@macheo.org"

    # Monday.com API Config
    BOARD_IDS = [
        "6642855163", "6642853462", "4325567636", "4033465087", "3722551002",
        "3722546433", "3722535164", "3722525919", "3722515533", "3722506503",
        "3722503006", "3722497489", "3722415867", "3722406184", "3722369659",
        "3722337116", "3722244889", "3722234267", "3722233437", "3722232187",
        "3721424836", "3712591089"
    ]

    def get_secrets():
        secret_name = os.environ.get('SECRET_NAME', 'macheo-reports-secrets')
        try:
            response = secrets_client.get_secret_value(SecretId=secret_name)
            secret = json.loads(response['SecretString'])
            return secret['API_KEY'], secret['SENDER_PASSWORD']
        except Exception as e:
            print(f"❌ Error retrieving secrets: {e}")
            raise e

    API_KEY, SENDER_PASSWORD = get_secrets()
    HEADERS = {"Authorization": API_KEY, "Content-Type": "application/json"}

    # Fetch the latest task ending with "Report-M & E Experiment" from the current year (with pagination)
    def fetch_latest_task(board_id):
        query = f"""
        {{
            boards(ids: [{board_id}]) {{
                id
                name
                items_page (limit: 500, cursor: null) {{
                    cursor
                    items {{
                        id
                        name
                        updated_at
                    }}
                }}
            }}
        }}
        """
        response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=HEADERS)
        data = response.json()

        if "errors" in data:
            print(f"❌ Error fetching board {board_id}: {data['errors']}")
            return None

        board = data.get("data", {}).get("boards", [])[0]
        if not board or "items_page" not in board:
            return None

        current_year = datetime.now().year
        latest_task = None
        items = board["items_page"]["items"]
        cursor = board["items_page"].get("cursor")

        for item in items:
            updated_at = datetime.strptime(item["updated_at"], "%Y-%m-%dT%H:%M:%SZ")
            if updated_at.year == current_year and item["name"].endswith("Report-M & E Experiment"):
                if not latest_task or updated_at > datetime.strptime(latest_task["updated_at"], "%Y-%m-%dT%H:%M:%SZ"):
                    latest_task = {
                        "board_name": board["name"],
                        "task_name": item["name"],
                        "task_id": item["id"],
                        "updated_at": item["updated_at"]
                    }

        while cursor:
            paginated_query = f"""
            {{
                next_items_page (limit: 500, cursor: "{cursor}") {{
                    cursor
                    items {{
                        id
                        name
                        updated_at
                    }}
                }}
            }}
            """
            response = requests.post("https://api.monday.com/v2", json={"query": paginated_query}, headers=HEADERS)
            data = response.json()
            if "errors" in data:
                print(f"❌ Pagination error for board {board_id}: {data['errors']}")
                break

            page = data.get("data", {}).get("next_items_page", {})
            items = page.get("items", [])
            cursor = page.get("cursor")

            for item in items:
                updated_at = datetime.strptime(item["updated_at"], "%Y-%m-%dT%H:%M:%SZ")
                if updated_at.year == current_year and item["name"].endswith("Report-M & E Experiment"):
                    if not latest_task or updated_at > datetime.strptime(latest_task["updated_at"], "%Y-%m-%dT%H:%M:%SZ"):
                        latest_task = {
                            "board_name": board["name"],
                            "task_name": item["name"],
                            "task_id": item["id"],
                            "updated_at": item["updated_at"]
                        }

        return latest_task

    # Fetch subitems & their files from a task
    def fetch_subitems(task_id):
        query = f"""
        {{
            items (ids: [{task_id}]) {{
                subitems {{
                    id
                    name
                    assets {{
                        id
                        name
                        public_url
                    }}
                }}
            }}
        }}
        """
        response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=HEADERS)
        data = response.json()
        
        if response.status_code == 200:
            print(f"✅ Successfully fetched subitems for task {task_id}.")
            return data.get("data", {}).get("items", [])[0].get("subitems", [])
        else:
            print(f"❌ API Error for task {task_id}: {response.text}")
            return []

    # Download file to memory
    def download_file_to_memory(file_url, file_name):
        response = requests.get(file_url)
        if response.status_code == 200:
            file_content = response.content
            if file_name.lower().endswith(".docx"):
                if file_content[:4] != b'PK\x03\x04':
                    print(f"⚠️ Invalid Word document: {file_name}")
                    return None
                print(f"📥 Downloaded DOCX: {file_name}")
                return io.BytesIO(file_content)
            elif file_name.lower().endswith((".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")):
                print(f"📥 Downloaded Image: {file_name}")
                return io.BytesIO(file_content)
        print(f"⚠️ Failed to download: {file_name}")
        return None

    # Check subitems for required files
    def check_task_files(task):
        subitems_data = fetch_subitems(task["task_id"])
        template_stream = None
        has_content = False
        downloaded_files = {}
        downloaded_images = {}

        print(f"\n📋 Checking subitems for task {task['task_id']} ({task['task_name']}):")
        if not subitems_data:
            print("  ❌ No subitems found!")
            return None, {}, {}

        for subitem in subitems_data:
            print(f"  Subitem: {subitem['name']} (ID: {subitem['id']})")
            if not subitem["assets"]:
                print("    - No files attached")
            for asset in subitem["assets"]:
                print(f"    - File: {asset['name']} (URL: {asset['public_url']})")
                file_stream = download_file_to_memory(asset["public_url"], asset["name"])
                if file_stream:
                    if asset["name"].lower().endswith(".docx"):
                        if "template" in asset["name"].lower() or "format" in subitem["name"].lower():
                            template_stream = file_stream
                            print(f"    ✅ Found potential template: {asset['name']}")
                        else:
                            doc = Document(file_stream)
                            text_content = "\n".join([p.text for p in doc.paragraphs])
                            downloaded_files[subitem["name"]] = text_content
                            has_content = True
                            print(f"    📄 Extracted text from {subitem['name']}")
                    elif asset["name"].lower().endswith((".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")):
                        downloaded_images[subitem["name"]] = file_stream
                        has_content = True
                        print(f"    🖼️ Stored image for {subitem['name']}")

        if not template_stream:
            print("  ❌ No template file found in subitems!")
        if not has_content:
            print("  ❌ No content files (DOCX or images) found in subitems!")
        
        return template_stream, downloaded_files, downloaded_images

    # Generate report
    def generate_report(template_doc, downloaded_files, downloaded_images, board_name):
        title_text = "January – December 2023 Individual health support <Donor name>"
        image_inserted = False

        print("📜 Template paragraphs before processing:")
        for i, para in enumerate(template_doc.paragraphs):
            print(f"  Para {i}: '{para.text.strip()}'")
        print("📥 Downloaded files available:", list(downloaded_files.keys()))
        print("📥 Downloaded images available:", list(downloaded_images.keys()))

        placeholders = [
            "Introduction", "Success", "Challenges", "…’s story", "The impact at moment of exit",
            "The long term impact", "Financial report", "The support our clients receive from Government",
            "Your specific contribution", "The waiting list", "Contact details"
        ]
        for para in template_doc.paragraphs:
            para_text = para.text.strip()
            if para_text in placeholders:
                for run in para.runs:
                    run.font.name = "Franklin Gothic Book"
                    run.font.size = Pt(18)
                    run.font.color.rgb = RGBColor(253, 103, 3)
                print(f"🎨 Formatted '{para_text}' to Franklin Gothic Book 18pt, #FD6703")

        if "Insert pic" in downloaded_images:
            for i, para in enumerate(template_doc.paragraphs):
                para_text = para.text.strip()
                if para_text == title_text:
                    new_para = template_doc.add_paragraph()
                    template_doc.paragraphs[i + 1]._element.addnext(new_para._element)
                    run = new_para.add_run()
                    run.add_picture(downloaded_images["Insert pic"], width=Inches(4))
                    print(f"🖼️ Inserted image after paragraph: {title_text}")
                    image_inserted = True
                    break
            
            if not image_inserted:
                for i, para in enumerate(template_doc.paragraphs):
                    if para.text.strip():
                        new_para = template_doc.add_paragraph()
                        template_doc.paragraphs[i + 1]._element.addnext(new_para._element)
                        run = new_para.add_run()
                        run.add_picture(downloaded_images["Insert pic"], width=Inches(4))
                        print(f"🖼️ Fallback: Inserted image after first non-empty paragraph: {para.text.strip()}")
                        image_inserted = True
                        break
        
        for para in template_doc.paragraphs[:]:
            section_title = para.text.strip()
            if section_title in downloaded_files:
                content_lines = downloaded_files[section_title].splitlines()
                if content_lines and content_lines[0].strip() == section_title:
                    text_content = "\n".join(content_lines[1:]).strip()
                else:
                    text_content = downloaded_files[section_title].strip()
                if text_content:
                    new_para = template_doc.add_paragraph(text_content)
                    para._element.addnext(new_para._element)
                    for run in new_para.runs:
                        run.font.name = "Tahoma"
                        run.font.size = Pt(10.5)
                        run.font.color.rgb = RGBColor(255, 255, 255)
                    print(f"✅ Appended text below: {section_title} (Tahoma 10.5pt, white)")
            if section_title in downloaded_images and section_title != "Insert pic":
                new_para = template_doc.add_paragraph()
                para._element.addnext(new_para._element)
                run = new_para.add_run()
                run.add_picture(downloaded_images[section_title], width=Inches(4))
                print(f"🖼️ Appended image below: {section_title}")

        print("📜 Template paragraphs after processing:")
        for i, para in enumerate(template_doc.paragraphs):
            print(f"  Para {i}: '{para.text.strip()}'")
        
        output_stream = io.BytesIO()
        template_doc.save(output_stream)
        output_stream.seek(0)
        return output_stream

    # Send email with report attachment
    def send_email_with_attachment(file_stream, board_name, task_id):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"Report_{board_name.replace(' ', '_')}_Task_{task_id}_{timestamp}.docx"
        
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL
        msg['Subject'] = f"Interventions Report for {board_name} - {timestamp}"

        body = f"Dear Recipient,\n\nAttached is the Interventions Report for {board_name} for January – December 2023.\n\nBest regards,\nMacheo Team"
        msg.attach(MIMEText(body, 'plain'))

        part = MIMEApplication(file_stream.read(), Name=file_name)
        part['Content-Disposition'] = f'attachment; filename="{file_name}"'
        msg.attach(part)

        try:
            server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
            server.quit()
            print(f"✅ Email sent to {RECIPIENT_EMAIL} with attachment: {file_name}")
            return file_name
        except Exception as e:
            print(f"❌ Error sending email for {board_name}: {e}")
            return None

    # Lambda handler
    def lambda_handler(event, context):
        latest_tasks = []
        for board_id in BOARD_IDS:
            task = fetch_latest_task(board_id)
            if task:
                latest_tasks.append(task)

        print(f"Found {len(latest_tasks)} tasks: {[task['board_name'] for task in latest_tasks]}")

        sent_emails = []

        for task in latest_tasks:
            print(f"\n📌 Board: {task['board_name']}")
            print(f"   ✅ Latest Task: {task['task_name']} (ID: {task['task_id']})")
            print(f"   🕒 Updated At: {task['updated_at']}")
            print("="*50)

            template_stream, downloaded_files, downloaded_images = check_task_files(task)
            
            if not template_stream or (not downloaded_files and not downloaded_images):
                print(f"❌ Skipping report generation for task {task['task_id']} due to missing template or content.")
                continue

            print(f"✅ Proceeding with report generation for task {task['task_id']}")
            template_doc = Document(template_stream)
            final_report_stream = generate_report(template_doc, downloaded_files, downloaded_images, task["board_name"])
            
            # Send email
            sent_file = send_email_with_attachment(final_report_stream, task["board_name"], task["task_id"])
            if sent_file:
                sent_emails.append(sent_file)
            
            print(f"✅ Finished task {task['task_id']}, moving to next...")

        response = {
            'statusCode': 200,
            'body': json.dumps({
                'sent_emails': sent_emails
            })
        }
        print(f"\nSent {len(sent_emails)} emails: {sent_emails}")
        return response

IndentationError: unexpected indent (2384024055.py, line 15)

In [1]:
import os
os.getcwd()


'C:\\Users\\Macheo'